In [3]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [10]:
from proj1_helpers import *
from data_analysis_logistic import *
from implementation import *
from cross_validation_reg_logistic import*

In [5]:
DATA_TRAIN_PATH = "data/train.csv" # download train data and supply path
y_tr, tX_tr, ids_tr = load_csv_data(DATA_TRAIN_PATH)

DATA_TEST_PATH = "data/test.csv" #download train data and supply path
y_fin, tX_fin, ids_fin = load_csv_data(DATA_TEST_PATH)

# Implementation

## Separation of the data

In [6]:
jet_num = 0
y0_tr, tX0_tr,y0_tr_new, tX0_tr_new, ids0_tr, y0_te, tX0_te, ids0_te, y0_fin, tX0_fin, ids0_fin = data_analysis(jet_num, y_tr, tX_tr, ids_tr, y_fin, tX_fin, ids_fin)
jet_num = 1
y1_tr, tX1_tr,y1_tr_new, tX1_tr_new, ids1_tr,y1_te, tX1_te, ids1_te, y1_fin, tX1_fin, ids1_fin = data_analysis(jet_num, y_tr, tX_tr, ids_tr, y_fin, tX_fin, ids_fin)
jet_num = 2
y2_tr, tX2_tr,y2_tr_new, tX2_tr_new, ids2_tr,y2_te, tX2_te, ids2_te, y2_fin, tX2_fin, ids2_fin = data_analysis(jet_num, y_tr, tX_tr, ids_tr, y_fin, tX_fin, ids_fin)
jet_num = 3
y3_tr, tX3_tr,y3_tr_new, tX3_tr_new, ids3_tr,y3_te, tX3_te, ids3_te, y3_fin, tX3_fin, ids3_fin = data_analysis(jet_num, y_tr, tX_tr, ids_tr, y_fin, tX_fin, ids_fin)

## Initialization

In [7]:
init_w0=np.zeros(tX0_tr.shape[1])
init_w1=np.zeros(tX1_tr.shape[1])
init_w2=np.zeros(tX2_tr.shape[1])
init_w3=np.zeros(tX3_tr.shape[1])

best_lambdas = [0., 0., 0., 0.,]

max_iters=40
gamma=0.01

## No cross validation - no extended feature - no separation of data

Résultats:
- Categorical accuracy: 0.663
- F1-Score: 0.362
- Our grade: 0.642

In [25]:
ratio=0.8
X_tr, X_te, Y_tr, Y_te, IDS_tr, IDS_te=split_data(tX_tr, y_tr, ids_tr, ratio, seed=1)
X_tr, m, std = standardize(X_tr)
X_te = standardize_te(X_te, m, std)

init_w=np.zeros(X_tr.shape[1])
best_lambda=test_lambda_cst(Y_tr,X_tr,Y_te,X_te,init_w, max_iters, gamma)

print(best_lambda)

0.017433288221999882


In [26]:
w, loss = reg_logistic_regression(Y_tr, X_tr,init_w, best_lambda, max_iters, gamma)

y_pred = predict_labels(w, X_te)

res = np.where(Y_te[:,] == y_pred[:,], 1, 0)
grade = np.mean(res)

print(grade)

0.64178


In [8]:
init_w=np.zeros(tX_tr.shape[1])
w, loss = reg_logistic_regression(y_tr, tX_tr,init_w, best_lambda, max_iters, gamma)

y_pred = predict_labels(w, tX_fin)
ids_pred = ids_fin
OUTPUT_PATH = 'results_reg_logistic_nocross_noext_nosep.csv' 
create_csv_submission(ids_pred, y_pred, OUTPUT_PATH)

/home/emma-hoggett/git_workspace/Project1_ML/implementation.py:70: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(y*np.log(sigma)+(1-y)*np.log(1-sigma))+lambda_/2*np.dot(w.T,w)
/home/emma-hoggett/git_workspace/Project1_ML/implementation.py:70: RuntimeWarning: invalid value encountered in multiply
  loss = -np.mean(y*np.log(sigma)+(1-y)*np.log(1-sigma))+lambda_/2*np.dot(w.T,w)
/home/emma-hoggett/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:75: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


## Cross validation of lambda - No extended feature

Résultats:
- Categorical accuracy: 0.678
- F1-Score: 0.615
- Our grade: 0.675 - *0.682


A revoir: Les lambdas sont ils bien definies? Notamment pour lambda 0.

*pas encore soumis

In [11]:
best_lambdas[0]= cross_validation_lambda_reg_logistic(y0_tr, tX0_tr, max_iters, gamma)
best_lambdas[1]= cross_validation_lambda_reg_logistic(y1_tr, tX1_tr, max_iters, gamma)
best_lambdas[2]= cross_validation_lambda_reg_logistic(y2_tr, tX2_tr, max_iters, gamma)
best_lambdas[3]= cross_validation_lambda_reg_logistic(y3_tr, tX3_tr, max_iters, gamma)

print(best_lambdas)

NameError: name 'build_k_indices' is not defined

In [33]:
init_w0=np.zeros(tX0_tr_new.shape[1])
init_w1=np.zeros(tX1_tr_new.shape[1])
init_w2=np.zeros(tX2_tr_new.shape[1])
init_w3=np.zeros(tX3_tr_new.shape[1])


w0, loss0 = reg_logistic_regression(y0_tr_new, tX0_tr_new,init_w0, best_lambdas[0], max_iters, gamma)
w1, loss1 = reg_logistic_regression(y1_tr_new, tX1_tr_new,init_w1, best_lambdas[1], max_iters, gamma)
w2, loss2 = reg_logistic_regression(y2_tr_new, tX2_tr_new,init_w2, best_lambdas[2], max_iters, gamma)
w3, loss3 = reg_logistic_regression(y3_tr_new, tX3_tr_new,init_w3, best_lambdas[3], max_iters, gamma)

y0_pred = predict_labels(w0, tX0_te)
y1_pred = predict_labels(w1, tX1_te)
y2_pred = predict_labels(w2, tX2_te)
y3_pred = predict_labels(w3, tX3_te)


grades = [0., 0., 0., 0.]
res0 = np.where(y0_te[:,] == y0_pred[:,], 1, 0)
grades[0] = np.mean(res0)
res1 = np.where(y1_te[:,] == y1_pred[:,], 1, 0)
grades[1] = np.mean(res0)
res2 = np.where(y2_te[:,] == y2_pred[:,], 1, 0)
grades[2] = np.mean(res2)
res3 = np.where(y3_te[:,] == y3_pred[:,], 1, 0)
grades[3] = np.mean(res3)
grade = np.mean(grades)

print(grade)

0.6819202850397081


In [61]:
y0_pred = predict_labels(w0, tX0_fin)
y1_pred = predict_labels(w1, tX1_fin)
y2_pred = predict_labels(w2, tX2_fin)
y3_pred = predict_labels(w3, tX3_fin)

In [62]:
y_pred = np.concatenate([y0_pred, y1_pred, y2_pred, y3_pred])
ids_pred = np.concatenate([ids0_fin, ids1_fin, ids2_fin, ids3_fin])
OUTPUT_PATH = 'results_reg_logistic_cross_noext_sep.csv' 
create_csv_submission(ids_pred, y_pred, OUTPUT_PATH)

## Cross validation - Extended feature

Résultat:
- Categorical accuracy:0.755
- F1-Score: 0.534
- Our grade: 0.771

In [34]:
lambda_=0.022

deg0=cross_validation_deg_reg_logistic(y0_tr, tX0_tr, max_iters, gamma,lambda_)
deg1=cross_validation_deg_reg_logistic(y1_tr, tX1_tr, max_iters, gamma,lambda_)
deg2=cross_validation_deg_reg_logistic(y2_tr, tX2_tr, max_iters, gamma,lambda_)
deg3=cross_validation_deg_reg_logistic(y3_tr, tX3_tr, max_iters, gamma,lambda_)

In [35]:
tX0_tr,tX0_te,tX0_fin=build_poly_data(tX0_tr,tX0_te,tX0_fin,deg0)
tX1_tr,tX1_te,tX1_fin=build_poly_data(tX1_tr,tX1_te,tX1_fin,deg1)
tX2_tr,tX2_te,tX2_fin=build_poly_data(tX2_tr,tX2_te,tX2_fin,deg2)
tX3_tr,tX3_te,tX3_fin=build_poly_data(tX3_tr,tX3_te,tX3_fin,deg3)

In [36]:
best_lambdas[0]= cross_validation_lambda_reg_logistic(y0_tr, tX0_tr, max_iters, gamma)
best_lambdas[1]= cross_validation_lambda_reg_logistic(y1_tr, tX1_tr, max_iters, gamma)
best_lambdas[2]= cross_validation_lambda_reg_logistic(y2_tr, tX2_tr, max_iters, gamma)
best_lambdas[3]= cross_validation_lambda_reg_logistic(y3_tr, tX3_tr, max_iters, gamma)

print(best_lambdas)
print(deg0)
print(deg1)
print(deg2)
print(deg3)

[188.73918221350996, 108.2636733874054, 188.73918221350996, 188.73918221350996]
[1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[3 4 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [37]:
init_w0=np.zeros(tX0_tr.shape[1])
init_w1=np.zeros(tX1_tr.shape[1])
init_w2=np.zeros(tX2_tr.shape[1])
init_w3=np.zeros(tX3_tr.shape[1])

w0, loss0 = reg_logistic_regression(y0_tr, tX0_tr,init_w0, best_lambdas[0], max_iters, gamma)
w1, loss1 = reg_logistic_regression(y1_tr, tX1_tr,init_w1, best_lambdas[1], max_iters, gamma)
w2, loss2 = reg_logistic_regression(y2_tr, tX2_tr,init_w2, best_lambdas[2], max_iters, gamma)
w3, loss3 = reg_logistic_regression(y3_tr, tX3_tr,init_w3, best_lambdas[3], max_iters, gamma)

y0_pred = predict_labels(w0, tX0_te)
y1_pred = predict_labels(w1, tX1_te)
y2_pred = predict_labels(w2, tX2_te)
y3_pred = predict_labels(w3, tX3_te)


grades = [0., 0., 0., 0.]
res0 = np.where(y0_te[:,] == y0_pred[:,], 1, 0)
grades[0] = np.mean(res0)
res1 = np.where(y1_te[:,] == y1_pred[:,], 1, 0)
grades[1] = np.mean(res0)
res2 = np.where(y2_te[:,] == y2_pred[:,], 1, 0)
grades[2] = np.mean(res2)
res3 = np.where(y3_te[:,] == y3_pred[:,], 1, 0)
grades[3] = np.mean(res3)
grade = np.mean(grades)

print(grade)

0.7682963323245651


In [ ]:
y0_pred = predict_labels(w0, tX0_fin)
y1_pred = predict_labels(w1, tX1_fin)
y2_pred = predict_labels(w2, tX2_fin)
y3_pred = predict_labels(w3, tX3_fin)

In [ ]:
y_pred = np.concatenate([y0_pred, y1_pred, y2_pred, y3_pred])
ids_pred = np.concatenate([ids0_fin, ids1_fin, ids2_fin, ids3_fin])
OUTPUT_PATH = 'results_reg_logistic_cross_ext_sep.csv' 
create_csv_submission(ids_pred, y_pred, OUTPUT_PATH)